# タスク 1c: テキスト生成の実行 (converse api を使用)

このノートブックでは Bedrock の Converse API を使用して、モデルを呼び出します。
以下のシナリオなどは タスク 1a と同様です

---

このノートブックでは、サポートエンジニアから受けたカスタマーサービスの品質について否定的なフィードバックを提供した顧客に対して、大規模言語モデル (LLM) を使用してメール返信を生成する方法を学習します。このノートブックでは、顧客の以前のメールに基づいて、お礼のメモ付きのメールを生成します。Amazon Nova Lite モデルは、Boto3 クライアントで Amazon Bedrock API で使用します。

このタスクで使用されるプロンプトは、ゼロショットプロンプトと呼ばれます。ゼロショットプロンプトでは、タスクまたは目的の出力を平易な言葉で言語モデルに説明します。その後、モデルは事前にトレーニングされた知識と機能を使用して、提供されたプロンプトのみに基づいて応答を生成したり、タスクを完了したりします。

#### シナリオ
あなたはAnyCompanyのカスタマーサービスマネージャーの Bob です。お客様の中には、カスタマーサポートエンジニアによるサービスに満足していない方がいらっしゃり、サービスについて否定的なフィードバックをされています。今、あなたはそうしたお客様に対し、謙虚にサービスのご不満をお詫びし、信頼回復したいと考えています。LLMを活用することで、お客様との過去のメールのやり取りからお客様の感情を汲み取った上で、人間らしくパーソナライズされたメールを一括で生成する必要があります。

## タスク 1c.1: 環境のセットアップ

このタスクでは、環境をセットアップします。

In [2]:
#Create a service client by name using the default session.
import json
import os
import sys

import boto3
import botocore

module_path = ".."
sys.path.append(os.path.abspath(module_path))

bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))

## タスク 1c.2: テキストの生成

このタスクでは、Amazon Bedrock サービスがメールを生成するための入力を準備します。

In [3]:
# create the prompt
prompt_data = """
指示: AnyCompany のカスタマーサービスマネージャーである Bob から、当社のカスタマーサポートエンジニアが提供したサービスについて否定的なフィードバックを提供した顧客「John Doe」にメールを書いてください。"""

## タスク 1c.3: Amazon Nova Lite 言語モデルを呼び出す

このタスクでは、以前に作成したプロンプトに基づいてモデルが出力を生成する方法を調べます。

次に、Amazon Nova Lite を呼び出します。

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **注:** Amazon Nova Lite は、約 300kトークンのコンテキストウィンドウをサポートし、次のパラメータを受け入れます。:
- `modelId`: これは、Amazon Bedrock で使用できるさまざまな基盤モデルのモデル ARN です
- `messages`: ロールとコンテンツを持つメッセージオブジェクトの配列
- `inferenceConfig`: これらは、モデルが出力を生成する際に考慮するパラメータです。

invoke_model を実行する際に設定していた以下は不要です

- `accept`: 入力リクエストのタイプ
- `contentType`: 出力のコンテンツ タイプ

### 完全な出力生成
この API は同期リクエストのため、モデルによって出力全体が生成されるまで待機します。

## ToDo①

以下のリンクを参考にして、Converse API のリクエストを完成させてください

[BedrockRuntime / Client / converse](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html)  


In [4]:
#invoke model
modelId = 'amazon.nova-lite-v1:0' # change this to use a different version from the model provider
outputText = "\n"
try:

    response = bedrock_client.converse(
        messages= # ここにメッセージ用のコードを追記
        modelId=modelId,
        inferenceConfig = {
            "maxTokens": 8192,
            "stopSequences": [],
            "temperature": 0,
            "topP": 0.9
        }
    )
    outputText = response.get('output').get('message').get('content')[0].get('text')

except botocore.exceptions.ClientError as error:
    
    if error.response['Error']['Code'] == 'AccessDeniedException':
           print(f"\x1b[41m{error.response['Error']['Message']}\
                \nTo troubeshoot this issue please refer to the following resources.\
                 \nhttps://docs.aws.amazon.com/IAM/latest/UserGuide/troubleshoot_access-denied.html\
                 \nhttps://docs.aws.amazon.com/bedrock/latest/userguide/security-iam.html\x1b[0m\n")
        
    else:
        raise error


In [5]:
# Print the complete response from Nova Lite
print(outputText)

件名: お客様のフィードバックに対する回答と謝意

親愛なるJohn Doe様、

お世話になっております。AnyCompanyのカスタマーサービスマネージャーであるBobと申します。この度は、当社のカスタマーサポートエンジニアが提供したサービスについて貴重なフィードバックをいただき、誠にありがとうございました。

お客様の貴重な意見を真摯に受け止め、今後サービスの向上に努めてまいります。お客様が経験された問題点や不満点について、具体的にお聞かせいただければ幸いです。これにより、より効果的な改善策を講じることができます。

また、お客様にご不便やご迷惑をおかけしてしまったことを深くお詫び申し上げます。今後このようなことが起こらないよう、全力を尽くしてまいります。

今後ともAnyCompanyをよろしくお願いいたします。

敬具

Bob [お名前]
カスタマーサービスマネージャー
AnyCompany
[連絡先]


レスポンス付きストリーム アプローチは、モデルの出力をすばやく取得するのに役立ち、ユーザーが読みながらサービスが出力を完了できるようにします。これは、モデルに長いテキストを生成するように要求するユースケースに役立ちます。後で、生成されたすべてのチャンクを組み合わせて完全な出力を形成し、ユースケースに使用できます。